### Manualy create the input dataframes based on Chinese census data

In [1]:
using DataFrames
using SyntheticPopulation

In [2]:

#each individual and each household represent 100.000 individuals or households
SCALE = 0.0001 

#all values are based on China census data
individual_popoulation_size = 21890000

#individuals
marginal_ind_age_sex = DataFrame(
    sex = repeat(['M', 'F'], 18),
    age = repeat(2:5:87, inner = 2), 
    population = SCALE .* 10000 .* [52.6, 49.0, 48.5, 44.8, 33.6, 30.6, 34.6, 28.8, 71.6, 63.4, 99.6, 90.9, 130.9, 119.4, 110.8, 103.5, 83.8, 76.4, 84.2, 77.7, 84.2, 77.8, 82.8, 79.9, 67.7, 71.0, 56.9, 62.6, 31.5, 35.3, 18.5, 23.0, 15.2, 19.7, 12.5, 16.0]
    )

marginal_ind_sex_maritalstatus = DataFrame(
    sex = repeat(['M', 'F'], 4), 
    maritalstatus = repeat(["Never_married", "Married", "Divorced", "Widowed"], inner = 2), 
    population = SCALE .* [1679, 1611, 5859, 5774, 140, 206, 128, 426] ./ 0.00082
    )

marginal_ind_income = DataFrame(
    income = [25394, 44855, 63969, 88026, 145915], 
    population = repeat([individual_popoulation_size * SCALE / 5], 5)
    )

#households
household_total_population = 8230000
marginal_hh_size = DataFrame(
    hh_size = [1,2,3,4,5],
    population = Int.(round.(SCALE * household_total_population .* [0.299, 0.331, 0.217, 0.09, 0.063]))
    )
nothing #to avoid printing output

### Create dataframe of individals
Some individual types may have the population 0 because we are using the scale of 0.0001

In [3]:

#generation of dataframe of individuals
aggregated_individuals = generate_joint_distribution(marginal_ind_sex_maritalstatus, marginal_ind_income, marginal_ind_age_sex, config_file = "tutorial_notebooks/config_file.json")

┌ Info: Inconsistent target margins, converting `X` and `mar` to proportions. Margin totals: [1930.0, 2190.0]
└ @ ProportionalFitting C:\Users\plzurekma\.julia\packages\ProportionalFitting\gNJEu\src\ipf.jl:61
┌ Info: Converged in 1 iterations.
└ @ ProportionalFitting C:\Users\plzurekma\.julia\packages\ProportionalFitting\gNJEu\src\ipf.jl:130
┌ Info: Inconsistent target margins, converting `X` and `mar` to proportions. Margin totals: [1085.0, 917.0]
└ @ ProportionalFitting C:\Users\plzurekma\.julia\packages\ProportionalFitting\gNJEu\src\ipf.jl:61
┌ Info: Converged in 1 iterations.
└ @ ProportionalFitting C:\Users\plzurekma\.julia\packages\ProportionalFitting\gNJEu\src\ipf.jl:130
┌ Info: Inconsistent target margins, converting `X` and `mar` to proportions. Margin totals: [1110.0, 951.0]
└ @ ProportionalFitting C:\Users\plzurekma\.julia\packages\ProportionalFitting\gNJEu\src\ipf.jl:61
┌ Info: Converged in 1 iterations.
└ @ ProportionalFitting C:\Users\plzurekma\.julia\packages\Proportiona

Row,id,maritalstatus,income,sex,age,population
,Int64,String?,Int64?,Char?,Int64?,Int64
1,1,Divorced,25394,F,22,3
2,2,Married,25394,F,22,9
3,3,Never_married,25394,F,22,0
4,4,Widowed,25394,F,22,0
5,5,Divorced,44855,F,22,3
6,6,Married,44855,F,22,9
7,7,Never_married,44855,F,22,0
8,8,Widowed,44855,F,22,0
9,9,Divorced,63969,F,22,3


In [4]:
# extra information about the individuals
describe(aggregated_individuals[aggregated_individuals.:population .> 0,:])

Row,variable,mean,min,median,max,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Int64,Type
1,id,323.158,1,335.5,600,0,Int64
2,maritalstatus,,Divorced,,Widowed,40,"Union{Missing, String}"
3,income,73631.8,25394,63969.0,145915,0,"Union{Missing, Int64}"
4,sex,,F,,M,0,"Union{Missing, Char}"
5,age,47.1316,2,47.0,87,0,"Union{Missing, Int64}"
6,population,5.59211,1,3.0,19,0,Int64


### Create dataframe of households

In [5]:

#generation of dataframe of households
aggregated_households = generate_joint_distribution(marginal_hh_size)

Row,id,hh_size,population
,Int64,Int64,Int64
1,1,1,246
2,2,2,272
3,3,3,179
4,4,4,74
5,5,5,52


### Allocate individuals to households

In [6]:
model, allocation_values, disaggregated_individuals, disaggregated_households = assign_and_optimize_individuals_to_households!(aggregated_individuals, aggregated_households)

Total number of individuals: 2125
Total number of households: 823
Allocation started...
Creation of individual constraints started.


Adding individual constraints 100%|██████████████████████| Time: 0:00:00


Creation of individual constraints finished.


Preparing household constraints 1/3 100%|████████████████| Time: 0:00:00


Preparation for creation of household constraints started.


...

Adding household constraints. 100%|██████████████████████| Time: 0:00:01


Creation of household constraints finished
Optimization of allocation started.
Optimization completed.
Objective value: 1758.0


Disaggregating individuals 100%|█████████████████████████| Time: 0:00:00
Disaggregating households 100%|██████████████████████████| Time: 0:00:00


Allocation finished.

(A JuMP Model
Maximization problem with:
Variables: 1748875
Objective function type: JuMP.AffExpr
`JuMP.AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 1062 constraints
`JuMP.AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 882 constraints
`JuMP.AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 4109 constraints
`JuMP.VariableRef`-in-`MathOptInterface.ZeroOne`: 1748875 constraints
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: GLPK
Names registered in the model: allocation, [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], 2125×3 DataFrame
  Row │ id     agg_ind_id  household_id 
      │ Int64  Int64?      Int64?       
──────┼─────────────────────────────────
    1 │     1           1           519
    2 │     2           1           520
    3 │     3           1           521
    4 │     4           2           522
    5 │     5           2           523
    6 │     6           2           524
    7 │     7           

### Check the outputs of allocation

In [7]:
model

A JuMP Model
Maximization problem with:
Variables: 1748875
Objective function type: JuMP.AffExpr
`JuMP.AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 1062 constraints
`JuMP.AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 882 constraints
`JuMP.AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 4109 constraints
`JuMP.VariableRef`-in-`MathOptInterface.ZeroOne`: 1748875 constraints
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: GLPK
Names registered in the model: allocation

In [8]:
aggregated_individuals

Row,id,maritalstatus,income,sex,age,population
,Int64,String?,Int64?,Char?,Int64?,Int64
1,1,Divorced,25394,F,22,3
2,2,Married,25394,F,22,9
3,3,Never_married,25394,F,22,0
4,4,Widowed,25394,F,22,0
5,5,Divorced,44855,F,22,3
6,6,Married,44855,F,22,9
7,7,Never_married,44855,F,22,0
8,8,Widowed,44855,F,22,0
9,9,Divorced,63969,F,22,3


In [9]:
disaggregated_individuals

Row,id,agg_ind_id,household_id
,Int64,Int64?,Int64?
1,1,1,519
2,2,1,520
3,3,1,521
4,4,2,522
5,5,2,523
6,6,2,524
7,7,2,525
8,8,2,526
9,9,2,527


In [10]:
aggregated_households

Row,id,hh_size,population
,Int64,Int64,Int64
1,1,1,246
2,2,2,272
3,3,3,179
4,4,4,74
5,5,5,52


In [12]:
disaggregated_households

Row,id,agg_hh_id,head_id,partner_id,child1_id,child2_id,child3_id
,Int64,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?
1,1,1,202,missing,missing,missing,missing
2,2,1,202,missing,missing,missing,missing
3,3,1,202,missing,missing,missing,missing
4,4,1,206,missing,missing,missing,missing
5,5,1,206,missing,missing,missing,missing
6,6,1,206,missing,missing,missing,missing
7,7,1,206,missing,missing,missing,missing
8,8,1,206,missing,missing,missing,missing
9,9,1,210,missing,missing,missing,missing


### See the full output of allocation (join of disaggrgated_households with aggregated_individuals)

In [13]:
function join_and_rename!(df1::DataFrame, df2::DataFrame, column_name::Symbol)
    df_joined = leftjoin(df1, df2, on = column_name => :id, makeunique=true, matchmissing = :notequal)

    # Rename the new columns
    for col in names(df2)[2:end]  # Skip the id column
        rename!(df_joined, Symbol(col) => Symbol(replace(string(column_name), "_id" => "_"*col)))
    end

    return df_joined
end

# Apply the function to each id column in df1
id_columns = [:head_id, :partner_id, :child1_id, :child2_id, :child3_id]
disaggregated_households_joined = disaggregated_households
for column_name in id_columns
    disaggregated_households_joined = join_and_rename!(disaggregated_households_joined, aggregated_individuals, column_name)
end
disaggregated_households_joined

Row,id,agg_hh_id,head_id,partner_id,child1_id,child2_id,child3_id,head_maritalstatus,head_income,head_sex,head_age,head_population,partner_maritalstatus,partner_income,partner_sex,partner_age,partner_population,child1_maritalstatus,child1_income,child1_sex,child1_age,child1_population,child2_maritalstatus,child2_income,child2_sex,child2_age,child2_population,child3_maritalstatus,child3_income,child3_sex,child3_age,child3_population
,Int64,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,String?,Int64?,Char?,Int64?,Int64?,String?,Int64?,Char?,Int64?,Int64?,String?,Int64?,Char?,Int64?,Int64?,String?,Int64?,Char?,Int64?,Int64?,String?,Int64?,Char?,Int64?,Int64?
1,772,5,178,missing,309,309,309,Married,145915,F,62,11,missing,missing,missing,missing,missing,Divorced,63969,M,27,4,Divorced,63969,M,27,4,Divorced,63969,M,27,4
2,773,5,178,missing,309,311,312,Married,145915,F,62,11,missing,missing,missing,missing,missing,Divorced,63969,M,27,4,Never_married,63969,M,27,1,Widowed,63969,M,27,1
3,774,5,178,missing,313,313,313,Married,145915,F,62,11,missing,missing,missing,missing,missing,Divorced,88026,M,27,4,Divorced,88026,M,27,4,Divorced,88026,M,27,4
4,775,5,178,missing,313,315,316,Married,145915,F,62,11,missing,missing,missing,missing,missing,Divorced,88026,M,27,4,Never_married,88026,M,27,1,Widowed,88026,M,27,1
5,776,5,178,missing,317,317,317,Married,145915,F,62,11,missing,missing,missing,missing,missing,Divorced,145915,M,27,4,Divorced,145915,M,27,4,Divorced,145915,M,27,4
6,777,5,182,missing,317,319,320,Married,25394,F,67,9,missing,missing,missing,missing,missing,Divorced,145915,M,27,4,Never_married,145915,M,27,1,Widowed,145915,M,27,1
7,778,5,182,missing,321,321,321,Married,25394,F,67,9,missing,missing,missing,missing,missing,Divorced,25394,M,32,5,Divorced,25394,M,32,5,Divorced,25394,M,32,5
8,779,5,182,missing,321,321,323,Married,25394,F,67,9,missing,missing,missing,missing,missing,Divorced,25394,M,32,5,Divorced,25394,M,32,5,Never_married,25394,M,32,1
9,780,5,182,missing,324,325,325,Married,25394,F,67,9,missing,missing,missing,missing,missing,Widowed,25394,M,32,1,Divorced,44855,M,32,5,Divorced,44855,M,32,5


In [14]:
describe(disaggregated_households_joined)

Row,variable,mean,min,median,max,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Int64,Type
1,id,412.0,1,412.0,823,0,Int64
2,agg_hh_id,2.28797,1,2.0,5,0,"Union{Missing, Int64}"
3,head_id,178.262,22,138.0,430,0,"Union{Missing, Int64}"
4,partner_id,389.248,302,354.0,558,371,"Union{Missing, Int64}"
5,child1_id,153.354,1,109.0,429,518,"Union{Missing, Int64}"
6,child2_id,296.183,145,298.0,432,697,"Union{Missing, Int64}"
7,child3_id,363.654,309,357.0,433,771,"Union{Missing, Int64}"
8,head_maritalstatus,,Married,,Married,0,"Union{Missing, String}"
9,head_income,73596.6,25394,63969.0,145915,0,"Union{Missing, Int64}"


#### Extra tests to validate outputs
At the moment the first test fails because the age difference constraints are disabled because of the computing complexity

In [21]:
print("check that there is a proper age difference between children and parents (between 20 and 40)\n")
for parent in [:head_age, :partner_age]
    for child in [:child1_age, :child2_age, :child3_age]
        print(unique(collect(skipmissing(disaggregated_households_joined[!, parent] - disaggregated_households_joined[!, child]))))
        print("\n")
    end
end
print("\n\n")

print("check that for all assigned individuals, the value in column :population from aggregated_individuals is larger than 1\n")
for column in [:head_population, :partner_population, :child1_population, :child2_population, :child3_population]
    print(unique(collect(skipmissing(disaggregated_households_joined[!, column]))))
end

check that there is a proper age difference between children and parents (between 20 and 40)
[35, 40, 30, 25, 20, 15, 10, 0, -5, -10, -15, -20, -25, 5]
[35, 40, 30, 25, 20, 15, 10, 0, -5, -10, -15, -20, -25]
[35, 40, 30, 25, 20, 15, 10]
[30, 35, 40]
[30]
Union{}[]


check that for all assigned individuals, the value in column :population from aggregated_individuals is larger than 1
[11, 9, 5, 12, 14, 18, 16, 3, 2][2, 12, 10, 8, 5, 3, 14, 19, 16][4, 5, 1, 3, 2, 10, 9][4, 1, 5, 3, 2, 10][4, 1, 5, 3]